This is the CBC, AES-32 function implemented as a class. Sbox is the substitution matrix, InvSbox is the Inv Substituion matrix. Rest algo is as described in the report. Inspiration of code taken from :
https://github.com/bozhu/AES-Python and https://github.com/boppreh/aes/blob/master/tests.py

In [8]:
import numpy as np
Sbox = (
    0x63, 0x7C, 0x77, 0x7B, 0xF2, 0x6B, 0x6F, 0xC5, 0x30, 0x01, 0x67, 0x2B, 0xFE, 0xD7, 0xAB, 0x76,
    0xCA, 0x82, 0xC9, 0x7D, 0xFA, 0x59, 0x47, 0xF0, 0xAD, 0xD4, 0xA2, 0xAF, 0x9C, 0xA4, 0x72, 0xC0,
    0xB7, 0xFD, 0x93, 0x26, 0x36, 0x3F, 0xF7, 0xCC, 0x34, 0xA5, 0xE5, 0xF1, 0x71, 0xD8, 0x31, 0x15,
    0x04, 0xC7, 0x23, 0xC3, 0x18, 0x96, 0x05, 0x9A, 0x07, 0x12, 0x80, 0xE2, 0xEB, 0x27, 0xB2, 0x75,
    0x09, 0x83, 0x2C, 0x1A, 0x1B, 0x6E, 0x5A, 0xA0, 0x52, 0x3B, 0xD6, 0xB3, 0x29, 0xE3, 0x2F, 0x84,
    0x53, 0xD1, 0x00, 0xED, 0x20, 0xFC, 0xB1, 0x5B, 0x6A, 0xCB, 0xBE, 0x39, 0x4A, 0x4C, 0x58, 0xCF,
    0xD0, 0xEF, 0xAA, 0xFB, 0x43, 0x4D, 0x33, 0x85, 0x45, 0xF9, 0x02, 0x7F, 0x50, 0x3C, 0x9F, 0xA8,
    0x51, 0xA3, 0x40, 0x8F, 0x92, 0x9D, 0x38, 0xF5, 0xBC, 0xB6, 0xDA, 0x21, 0x10, 0xFF, 0xF3, 0xD2,
    0xCD, 0x0C, 0x13, 0xEC, 0x5F, 0x97, 0x44, 0x17, 0xC4, 0xA7, 0x7E, 0x3D, 0x64, 0x5D, 0x19, 0x73,
    0x60, 0x81, 0x4F, 0xDC, 0x22, 0x2A, 0x90, 0x88, 0x46, 0xEE, 0xB8, 0x14, 0xDE, 0x5E, 0x0B, 0xDB,
    0xE0, 0x32, 0x3A, 0x0A, 0x49, 0x06, 0x24, 0x5C, 0xC2, 0xD3, 0xAC, 0x62, 0x91, 0x95, 0xE4, 0x79,
    0xE7, 0xC8, 0x37, 0x6D, 0x8D, 0xD5, 0x4E, 0xA9, 0x6C, 0x56, 0xF4, 0xEA, 0x65, 0x7A, 0xAE, 0x08,
    0xBA, 0x78, 0x25, 0x2E, 0x1C, 0xA6, 0xB4, 0xC6, 0xE8, 0xDD, 0x74, 0x1F, 0x4B, 0xBD, 0x8B, 0x8A,
    0x70, 0x3E, 0xB5, 0x66, 0x48, 0x03, 0xF6, 0x0E, 0x61, 0x35, 0x57, 0xB9, 0x86, 0xC1, 0x1D, 0x9E,
    0xE1, 0xF8, 0x98, 0x11, 0x69, 0xD9, 0x8E, 0x94, 0x9B, 0x1E, 0x87, 0xE9, 0xCE, 0x55, 0x28, 0xDF,
    0x8C, 0xA1, 0x89, 0x0D, 0xBF, 0xE6, 0x42, 0x68, 0x41, 0x99, 0x2D, 0x0F, 0xB0, 0x54, 0xBB, 0x16,
)

InvSbox = (
    0x52, 0x09, 0x6A, 0xD5, 0x30, 0x36, 0xA5, 0x38, 0xBF, 0x40, 0xA3, 0x9E, 0x81, 0xF3, 0xD7, 0xFB,
    0x7C, 0xE3, 0x39, 0x82, 0x9B, 0x2F, 0xFF, 0x87, 0x34, 0x8E, 0x43, 0x44, 0xC4, 0xDE, 0xE9, 0xCB,
    0x54, 0x7B, 0x94, 0x32, 0xA6, 0xC2, 0x23, 0x3D, 0xEE, 0x4C, 0x95, 0x0B, 0x42, 0xFA, 0xC3, 0x4E,
    0x08, 0x2E, 0xA1, 0x66, 0x28, 0xD9, 0x24, 0xB2, 0x76, 0x5B, 0xA2, 0x49, 0x6D, 0x8B, 0xD1, 0x25,
    0x72, 0xF8, 0xF6, 0x64, 0x86, 0x68, 0x98, 0x16, 0xD4, 0xA4, 0x5C, 0xCC, 0x5D, 0x65, 0xB6, 0x92,
    0x6C, 0x70, 0x48, 0x50, 0xFD, 0xED, 0xB9, 0xDA, 0x5E, 0x15, 0x46, 0x57, 0xA7, 0x8D, 0x9D, 0x84,
    0x90, 0xD8, 0xAB, 0x00, 0x8C, 0xBC, 0xD3, 0x0A, 0xF7, 0xE4, 0x58, 0x05, 0xB8, 0xB3, 0x45, 0x06,
    0xD0, 0x2C, 0x1E, 0x8F, 0xCA, 0x3F, 0x0F, 0x02, 0xC1, 0xAF, 0xBD, 0x03, 0x01, 0x13, 0x8A, 0x6B,
    0x3A, 0x91, 0x11, 0x41, 0x4F, 0x67, 0xDC, 0xEA, 0x97, 0xF2, 0xCF, 0xCE, 0xF0, 0xB4, 0xE6, 0x73,
    0x96, 0xAC, 0x74, 0x22, 0xE7, 0xAD, 0x35, 0x85, 0xE2, 0xF9, 0x37, 0xE8, 0x1C, 0x75, 0xDF, 0x6E,
    0x47, 0xF1, 0x1A, 0x71, 0x1D, 0x29, 0xC5, 0x89, 0x6F, 0xB7, 0x62, 0x0E, 0xAA, 0x18, 0xBE, 0x1B,
    0xFC, 0x56, 0x3E, 0x4B, 0xC6, 0xD2, 0x79, 0x20, 0x9A, 0xDB, 0xC0, 0xFE, 0x78, 0xCD, 0x5A, 0xF4,
    0x1F, 0xDD, 0xA8, 0x33, 0x88, 0x07, 0xC7, 0x31, 0xB1, 0x12, 0x10, 0x59, 0x27, 0x80, 0xEC, 0x5F,
    0x60, 0x51, 0x7F, 0xA9, 0x19, 0xB5, 0x4A, 0x0D, 0x2D, 0xE5, 0x7A, 0x9F, 0x93, 0xC9, 0x9C, 0xEF,
    0xA0, 0xE0, 0x3B, 0x4D, 0xAE, 0x2A, 0xF5, 0xB0, 0xC8, 0xEB, 0xBB, 0x3C, 0x83, 0x53, 0x99, 0x61,
    0x17, 0x2B, 0x04, 0x7E, 0xBA, 0x77, 0xD6, 0x26, 0xE1, 0x69, 0x14, 0x63, 0x55, 0x21, 0x0C, 0x7D,
)

Rcon = (
    0x00, 0x01, 0x02, 0x04, 0x08, 0x10, 0x20, 0x40,
    0x80, 0x1B, 0x36, 0x6C, 0xD8, 0xAB, 0x4D, 0x9A,
    0x2F, 0x5E, 0xBC, 0x63, 0xC6, 0x97, 0x35, 0x6A,
    0xD4, 0xB3, 0x7D, 0xFA, 0xEF, 0xC5, 0x91, 0x39,
)


def text2matrix(text):
    matrix = []
    for i in range(4):
        byte = (text >> (8 * (3 - i))) & 0xFF
        if i % 2 == 0:
            matrix.append([byte])
        else:
            matrix[int(i/2)].append(byte)
    return matrix

def matrix2text(matrix):
    text = 0
    for i in range(2):
        for j in range(2):
            text |= (matrix[i][j] << (24 - 8 * (2 * i + j)))
    return text


class AES_32:
    def __init__(self, master_key):
        self.change_key(master_key)

    def change_key(self, master_key):
        self.round_keys = text2matrix(master_key)

        for i in range(2, 2 * 8):
            self.round_keys.append([])
            if i % 2 == 0:
                byte = self.round_keys[i - 2][0]        \
                     ^ Sbox[self.round_keys[i - 1][1]]  \
                     ^ Rcon[int(i /2)]
                self.round_keys[i].append(byte)

                for j in range(1, 2):
                    byte = self.round_keys[i - 2][j]    \
                         ^ Sbox[self.round_keys[i - 1][(j + 1) % 2]]
                    self.round_keys[i].append(byte)
            else:
                for j in range(2):
                    byte = self.round_keys[i - 2][j]    \
                         ^ self.round_keys[i - 1][j]
                    self.round_keys[i].append(byte)

    def encrypt(self, inp_msg):
        self.plain_state = text2matrix(inp_msg)

        self.__add_round_key(self.plain_state, self.round_keys[:2])

        for i in range(1, 7):
            self.__round_encrypt(self.plain_state, self.round_keys[2 * i : 2 * (i + 1)])

        self.__sub_bytes(self.plain_state)
        self.__shift_rows(self.plain_state)
        self.__add_round_key(self.plain_state, self.round_keys[14:])

        return matrix2text(self.plain_state)

    def decrypt(self, ciphertext):
        self.cipher_state = text2matrix(ciphertext)

        self.__add_round_key(self.cipher_state, self.round_keys[14:])

        for i in range(6, 0, -1):
            self.__round_decrypt(self.cipher_state, self.round_keys[2 * i : 2 * (i + 1)])
        
        self.__inv_shift_rows(self.cipher_state) 
        self.__inv_sub_bytes(self.cipher_state)
        self.__add_round_key(self.cipher_state, self.round_keys[:2])

        return matrix2text(self.cipher_state)

    def __add_round_key(self, s, k):
        for i in range(2):
            for j in range(2):
                s[i][j] ^= k[i][j]


    def __round_encrypt(self, state_matrix, key_matrix):
        self.__sub_bytes(state_matrix)
        self.__shift_rows(state_matrix)
        self.__mix_columns(state_matrix)
        self.__add_round_key(state_matrix, key_matrix)


    def __round_decrypt(self, state_matrix, key_matrix):
        self.__inv_shift_rows(state_matrix)
        self.__inv_sub_bytes(state_matrix)
        self.__add_round_key(state_matrix, key_matrix)
        self.__inv_mix_columns(state_matrix)

    def __sub_bytes(self, s):
        for i in range(2):
            for j in range(2):
                s[i][j] = Sbox[s[i][j]]


    def __inv_sub_bytes(self, s):
        for i in range(2):
            for j in range(2):
                s[i][j] = InvSbox[s[i][j]]


    def __shift_rows(self, s):
        s[1][0], s[1][1] = s[1][1], s[1][0]


    def __inv_shift_rows(self, s):
        s[1][0], s[1][1] = s[1][1], s[1][0]



    def __mix_columns(self, s):
        s = np.matrix(s)
        M = np.matrix(([2,3],[1,2]))
        return M.dot(s)


    def __inv_mix_columns(self, s):
        s = np.matrix(s)
        M = np.matrix(([2,253],[255,2]))
        return M.dot(s)

        self.__mix_columns(s)

class AES_TEST_32():
    def setUp(self, master_key):
        self.AES_32 = AES_32(master_key)

    def test_encryption(self, inp_msg):
        encrypted = self.AES_32.encrypt(inp_msg)
        return encrypted

    def test_decryption(self, ciphertext):
        decrypted = self.AES_32.decrypt(ciphertext)
        return decrypted

def pad(plaintext):
    padding_len = 4 - (len(plaintext) % 4)
    padding = bytes([padding_len] * padding_len)
    return plaintext + padding

def split_blocks(message, block_size=4, require_padding=True):
    return [message[i:i+4] for i in range(0, len(message), block_size)]

def unpad(plaintext):
    padding_len = plaintext[-1]
    message, padding = plaintext[:-padding_len], plaintext[-padding_len:]
    return message

def xor_bytes(a, b):
    return bytes(i^j for i, j in zip(a, b))

def CBC_enc(plaintxt, key, iv):
  aes_32 = AES_TEST_32()
  aes_32.setUp(master_key = key)
  blocks = []
  previous = iv
  i = 0
  for plaintext_block in split_blocks(plaintxt):
    plaintext_block = pad(plaintext_block)
    test = int.from_bytes((xor_bytes(plaintext_block, previous)), byteorder = 'big')
    block = aes_32.test_encryption(inp_msg = test).to_bytes(4, 'big')
    blocks.append(block)
    previous = block
    i = i+1
  return b''.join(blocks)

def CBC_dec(ciphertxt, key, iv):
  aes_32 = AES_TEST_32()
  aes_32.setUp(master_key = key)
  blocks = []
  previous = iv
  i = 0
  for ciphertext_block in split_blocks(ciphertxt):
    block = aes_32.test_decryption(ciphertext = int.from_bytes(ciphertext_block, 'big')).to_bytes(4, 'big')
    block = xor_bytes(block, previous)
    blocks.append(block)
    previous = ciphertext_block
    i = i+1
  str_out = b''.join(blocks)
  str_out = unpad(str_out)
  return str_out

def Check_CBC_enc_dec(plaintext, key, iv):
  ciphertxt = CBC_enc(plaintext, key, iv)
  print("Cipher text: ", ciphertxt)
  dec_txt = CBC_dec(ciphertxt, key, iv)
  print("Decrypted text: ", dec_txt)
  if(dec_txt == plaintext):
    print("Algorithm Working Fine")


In [9]:
Check_CBC_enc_dec(b'This is my message to be encrypted, you can type anything instead of this', key = 0x2b7e1516, iv = b'love')

Cipher text:  b'\xd3\x83\xce.4R\xf7\xf4\xda9\x0b\xc8\x92r\xd2\xe8Z@\xe1\x8d\x11Z\xecSq\xc3\t\xb0t\x06\xd3\x82\x9d\xad\xa1\xff\xeb\x90z\xe5\x01\xf9\xb0-\xfc<h\x04K\xf0x\xfa\xcbkn5\x9b\xf5t\xd9\xc2K\x0653\xcdm\xbe\x9a\xea\xee\xe9\x85\xe5k\x99'
Decrypted text:  b'This is my message to be encrypted, you can type anything instead of this'
Algorithm Working Fine
